In [1]:
d = read.csv("../data/wiki/user_data.csv", header = TRUE)
library(lsr)
head(d)

user_id,edit_count,gender,admin_ascention,admin,eigen_central,eigen_central_bin,community,coord_given_all,coord_received_all,coord_given_ingroup,coord_received_ingroup,post_count,italics_freq,bold_freq,link_freq,function_words_freq,avg_length_tokens
RSOVRGR,2,unknown,,False,4.891008e-06,False,1,NaN,NaN,NA,NA,1,NA,NA,NA,16.000000,1.000000
Athenagoras,24,unknown,,False,2.640434e-08,False,0,0,0,NA,NA,1,NA,NA,NA,16.000000,4.000000
Themaxviwe,1465,male,,False,1.393723e-04,False,1,0,0,NA,NA,3,NA,NA,NA,6.333333,1.000000
Cloud02,765,unknown,,False,4.705397e-06,False,3,0,0,NA,NA,3,NA,NA,NA,21.333333,3.000000
Anthony Winward,10669,male,,False,3.804158e-05,False,4,NaN,NaN,NA,NA,7,NA,NA,NA,5.571429,1.142857
Seikku Kaita,81,unknown,,False,NA,,NA,NaN,NaN,NA,NA,1,NA,NA,NA,7.000000,1.000000


In [38]:
cor.test(d$eigenvector_centrality, d$coord_received_univ_agg, alternative="greater", method="spearman", exact = FALSE)


	Spearman's rank correlation rho

data:  d$eigenvector_centrality and d$coord_received_univ_agg
S = 8.9218e+11, p-value < 2.2e-16
alternative hypothesis: true rho is greater than 0
sample estimates:
     rho 
0.189738 


In [39]:
cor.test(d$eigenvector_centrality, d$coord_given_univ_agg, alternative="greater", method="spearman", exact = FALSE)


	Spearman's rank correlation rho

data:  d$eigenvector_centrality and d$coord_given_univ_agg
S = 3.5665e+11, p-value < 2.2e-16
alternative hypothesis: true rho is greater than 0
sample estimates:
      rho 
0.1941795 


In [46]:
cor.test(d$coord_received, d$link_usage)


	Pearson's product-moment correlation

data:  d$coord_received and d$link_usage
t = -0.62709, df = 20649, p-value = 0.5306
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 -0.018001829  0.009275628
sample estimates:
         cor 
-0.004363912 


In [2]:
t.test(d$coord_given_all, d$coord_given_ingroup)


	Welch Two Sample t-test

data:  d$coord_given_all and d$coord_given_ingroup
t = 2.1289, df = 74.774, p-value = 0.03656
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.0006545127 0.0197317493
sample estimates:
   mean of x    mean of y 
 0.005561945 -0.004631186 


In [3]:
t.test(d$coord_received_all, d$coord_received_ingroup)


	Welch Two Sample t-test

data:  d$coord_received_all and d$coord_received_ingroup
t = 2.1289, df = 74.774, p-value = 0.03656
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.0006545127 0.0197317493
sample estimates:
   mean of x    mean of y 
 0.005561945 -0.004631186 
